## Members

# Pre-processing

In [56]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

from vncorenlp import VnCoreNLP

In [57]:
vncorenlp_file = "VnCoreNLP/VnCoreNLP-1.1.1.jar"

In [58]:
data_df = pd.read_csv("vfnd/CSV/vn_news_223_tdlfr.csv")
data_df.head()

,text,domain,label
0,Thủ tướng Abe cúi đầu xin lỗi vì hành động phi...,binhluan.biz,1
1,Thủ tướng Nhật cúi đầu xin lỗi vì tinh thần ph...,www.ipick.vn,1
2,Choáng! Cơ trưởng đeo khăn quàng quẩy banh nóc...,tintucqpvn.net,1
3,Chưa bao giờ nhạc Kpop lại dễ hát đến thế!!!\n...,tintucqpvn.net,1
4,"Đại học Hutech sẽ áp dụng cải cách ""Tiếq Việt""...",www.gioitreviet.net,1


In [59]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    223 non-null    object
 1   domain  223 non-null    object
 2   label   223 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 5.4+ KB


#### List of vietnamese stopwords

In [60]:
vn_stopwords = []
with open('data/vietnamese_stopwords.txt', encoding="utf8") as file:
    for line in file.read().splitlines():
        vn_stopwords.append(line.strip())

#### Dữ liệu có bị lặp không

In [61]:
data_df.index.duplicated().sum()

0

In [62]:
data_df.isnull().sum()

text      0
domain    0
label     0
dtype: int64

In [63]:
test_str = data_df['text'][2]
test_str

'Choáng! Cơ trưởng đeo khăn quàng quẩy banh nóc trên sân khấu trường!!!\nXót xa hình ảnh thầy trò lội bùn băng rừng đến trường ở Nghệ An\nSau lũ, nhiều học sinh vùng cao và giáo viên ở Nghệ An phải đối mặt với thiếu thốn vì giao thông bị chia cắt, cơ sở vật chất bị lũ cuốn trôi.\nDưới đây là một số hình ảnh học sinh và giáo viên ở Nghệ An lội bùn băng rừng đến trường:\nXã biên giới Mường Típ, Mường Ải, huyện miền núi Kỳ Sơn, Nghệ An là một trong những nơi đã bị lũ dữ ập đến. Nhiều đoạn đường, bị hàng ngàn khối đất đá sạt lở tràn lấp khiến nơi đây bị cô lập.\nNhiều em học sinh phải lội bùn, băng rừng để đến trường trong hoàn cảnh khó khăn.\nThầy Nguyễn Công Danh, Hiệu trưởng Trường Phổ thông dân tộc bán trú THCS Nập Típ (huyện Kỳ Sơn) cho biết, đến thời điểm hiện tại, các giáo viên của trường mới chỉ vận động được 184/387 học sinh THCS đến trường học và 180/197 học sinh ở bậc tiểu học đến trường học trở lại.\nĐể đến trường với đoạn đường chưa đến 2km, nhiều em học sinh nơi đây phải trầy

In [64]:
test_str.lower()

'choáng! cơ trưởng đeo khăn quàng quẩy banh nóc trên sân khấu trường!!!\nxót xa hình ảnh thầy trò lội bùn băng rừng đến trường ở nghệ an\nsau lũ, nhiều học sinh vùng cao và giáo viên ở nghệ an phải đối mặt với thiếu thốn vì giao thông bị chia cắt, cơ sở vật chất bị lũ cuốn trôi.\ndưới đây là một số hình ảnh học sinh và giáo viên ở nghệ an lội bùn băng rừng đến trường:\nxã biên giới mường típ, mường ải, huyện miền núi kỳ sơn, nghệ an là một trong những nơi đã bị lũ dữ ập đến. nhiều đoạn đường, bị hàng ngàn khối đất đá sạt lở tràn lấp khiến nơi đây bị cô lập.\nnhiều em học sinh phải lội bùn, băng rừng để đến trường trong hoàn cảnh khó khăn.\nthầy nguyễn công danh, hiệu trưởng trường phổ thông dân tộc bán trú thcs nập típ (huyện kỳ sơn) cho biết, đến thời điểm hiện tại, các giáo viên của trường mới chỉ vận động được 184/387 học sinh thcs đến trường học và 180/197 học sinh ở bậc tiểu học đến trường học trở lại.\nđể đến trường với đoạn đường chưa đến 2km, nhiều em học sinh nơi đây phải trầy

In [65]:
# Remove links
test_str = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", test_str)
# Remove all number in string
test_str = ''.join([i for i in test_str if not i.isdigit()])
# Remove all special characters and punctuation
test_str = re.sub('\W+',' ', test_str)
test_str = test_str.strip()
test_str

'Choáng Cơ trưởng đeo khăn quàng quẩy banh nóc trên sân khấu trường Xót xa hình ảnh thầy trò lội bùn băng rừng đến trường ở Nghệ An Sau lũ nhiều học sinh vùng cao và giáo viên ở Nghệ An phải đối mặt với thiếu thốn vì giao thông bị chia cắt cơ sở vật chất bị lũ cuốn trôi Dưới đây là một số hình ảnh học sinh và giáo viên ở Nghệ An lội bùn băng rừng đến trường Xã biên giới Mường Típ Mường Ải huyện miền núi Kỳ Sơn Nghệ An là một trong những nơi đã bị lũ dữ ập đến Nhiều đoạn đường bị hàng ngàn khối đất đá sạt lở tràn lấp khiến nơi đây bị cô lập Nhiều em học sinh phải lội bùn băng rừng để đến trường trong hoàn cảnh khó khăn Thầy Nguyễn Công Danh Hiệu trưởng Trường Phổ thông dân tộc bán trú THCS Nập Típ huyện Kỳ Sơn cho biết đến thời điểm hiện tại các giáo viên của trường mới chỉ vận động được học sinh THCS đến trường học và học sinh ở bậc tiểu học đến trường học trở lại Để đến trường với đoạn đường chưa đến km nhiều em học sinh nơi đây phải trầy trựợt suốt hơn giờ đồng hồ Các học sinh và giá

In [66]:
# Remove stop words
filtered_word = [w for w in test_str.split(' ') if not w in vn_stopwords]
filtered_str = ' '.join(filtered_word)
filtered_str

'Choáng Cơ trưởng đeo khăn quàng quẩy banh sân khấu trường Xót hình ảnh thầy trò lội bùn băng rừng trường Nghệ An Sau lũ học sinh giáo viên Nghệ An đối mặt thốn giao thông chia cắt sở vật chất lũ trôi Dưới hình ảnh học sinh giáo viên Nghệ An lội bùn băng rừng trường Xã biên giới Mường Típ Mường Ải huyện miền núi Kỳ Sơn Nghệ An lũ ập Nhiều đoạn đường hàng ngàn khối đất đá sạt lở tràn lấp lập Nhiều học sinh lội bùn băng rừng trường hoàn cảnh khăn Thầy Nguyễn Công Danh Hiệu trưởng Trường Phổ thông dân tộc trú THCS Nập Típ huyện Kỳ Sơn thời hiện giáo viên trường vận động học sinh THCS trường học học sinh bậc tiểu học trường học trở Để trường đoạn đường km học sinh trầy trựợt suốt đồng hồ Các học sinh giáo viên lũ đi giúp đỡ khăn Nguồn'

In [76]:
# annotator = VnCoreNLP(vncorenlp_file, annotators="wseg,pos,ner,parse", max_heap_size='-Xmx2g')
annotator = VnCoreNLP(vncorenlp_file)
with annotator:
    tokenized_text = annotator.tokenize(filtered_str)[0]
    print(len(tokenized_text))
    print(' '.join(tokenized_text))

111
Choáng_Cơ trưởng đeo khăn_quàng quẩy banh sân_khấu trường Xót hình_ảnh thầy_trò lội bùn băng rừng trường Nghệ_An Sau lũ học_sinh giáo_viên Nghệ_An đối_mặt thốn giao_thông chia_cắt sở vật_chất lũ trôi Dưới hình_ảnh học_sinh giáo_viên Nghệ_An lội bùn băng rừng trường Xã biên_giới Mường_Típ_Mường_Ải huyện miền núi Kỳ_Sơn Nghệ_An lũ ập Nhiều đoạn đường hàng ngàn khối đất_đá sạt_lở tràn lấp lập Nhiều học_sinh lội bùn băng rừng trường hoàn_cảnh khăn Thầy Nguyễn_Công_Danh_Hiệu trưởng Trường Phổ_thông dân_tộc trú THCS Nập_Típ huyện Kỳ_Sơn thời hiện giáo_viên trường vận_động học_sinh THCS trường_học học_sinh bậc tiểu_học trường_học trở Để trường_đoạn đường km học_sinh trầy trựợt suốt đồng_hồ Các học_sinh giáo_viên lũ đi giúp_đỡ khăn Nguồn


In [75]:
from sklearn.feature_extraction.text import CountVectorizer

# Use CountVectorizer to convert text data into numerical values
cv = CountVectorizer(analyzer='word')
data = cv.fit_transform(tokenized_text)
df_data = pd.DataFrame(data.toarray(), columns=cv.get_feature_names())
df_data.head()

,banh,biên_giới,bùn,băng,bậc,chia_cắt,choáng_cơ,các,dân_tộc,dưới,...,xót,đeo,đi,đoạn,đường,đất_đá,để,đối_mặt,đồng_hồ,ập
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Importing wordcloud for plotting word clouds and textwrap for wrapping longer text
from wordcloud import WordCloud
from textwrap import wrap

# Function for generating word clouds
def generate_wordcloud(data,title):
  wc = WordCloud(width=400, height=330, max_words=150,colormap="Dark2").generate_from_frequencies(data)
  plt.figure(figsize=(10,8))
  plt.imshow(wc, interpolation='bilinear')
  plt.axis("off")
#   plt.title('\n'.join(wrap(title,60)),fontsize=13)
  plt.show()
  
# Transposing document term matrix
df_data = df_data.transpose()

# Plotting word cloud for each product
for index,product in enumerate(df_data.columns):
    print(index, product)
    generate_wordcloud(df_data[product].sort_values(ascending=False),product)

### EDA

### Build model

### Deploy